# Text Analytics Coursework -- Tasks 1.1 to 1.3

# Instructions:

This notebook is forms the first part of your coursework assignment for Text Analytics in Spring 2025. You will need to read the instructions below and complete numbered tasks indicated by "TASK n". To complete the tasks, you will write code or explanations between the comments "#WRITE YOUR ANSWER HERE" and "#END OF ANSWER". For example:

TASK 0: Complete the function below to output "hello world". 


In [7]:
def demo_fun():
    # WRITE YOUR ANSWER HERE
    print("hello world")
    # END OF ANSWER

### DO NOT MODIFY
demo_fun()
###

hello world


There is also some code in the cell that should not be modified. This code saves your results to file in the correct format, which is necessary for us to be able to mark your answers. Before you submit your notebook, please make sure this code has not been modified, then restart your kernel, clear all cell outputs, run all of your code once again, then save the notebook. 

Please note:
  * The notebook you upload must include all the saved cell output after running all cells.
  * The notebook code must be complete so that it reproduces all your output when we run it. 
  * For this coursework, we recommend that you use your virtual environment that you created for the labs. The packages you need are: numpy, scipy, nltk, pytorch, transformers and datasets (from HuggingFace), pandas, matplotlib and scikit-learn. 

## Marking guidelines:
1. This notebook is worth 32% of the marks for the Text Analytics assignment.
1. The number of marks for each task is shown alongside the task.
1. We will evaluate the output of your code after running it, and marks will be awarded based on how well the output matches the task's instructions. 
1. We will give partial marks for incomplete or partially correct answers. 
1. We do not give additional marks for code style or comments, but clear code will help us to understand what you have done so that we can award partial marks where necessary. 
1. Unless the task asks you to implement something from scratch, there is no penalty for using software libraries in your implementation.

## Support:

The main source of support will be during the lab sessions. The TAs and lecturers will help you with questions about the lectures, the code provided for you in this notebook, and general questions about the topics we cover. For the assessed tasks, they can only answer clarifying questions about what you have to do. Please email Edwin if you have any other queries edwin.simpson@bristol.ac.uk and/or post your query to the Teams channel for this unit.

## Deadline:

The notebook must be submitted along with the second notebook on Blackboard before **Monday 28th April at 13.00**. 

## Submission:

For this part of the assignment, please zip up the folder containing this file and the 'outputs' directory, containing the output from this notebook as .csv files. Please name your notebook file like this:
   * Name this notebook 'text_analytics_part1_\<student number\>.ipynb'. Replace '\<student number\>' with your student number, which consists only of digits beginning with '2'. 
   * We mark anonymously, so please don't include your name in the notebook.

You can submit the file on Blackboard to the submission point "Text Analytics Part 1 Notebook". Remember that the assignment also has parts 2 and 3, described in the PDF file on Blackboard.

# Setup: random seeds

Each student will work with slightly different data splits and model weights, which will be determined by setting your 'random seed'. 
We will check that your results come from using your random seed. Please set the seed in the cell below by changing the value of 'my_student_number' to your own student number (not your username, the number you can see on eVision that contains only digits). 

Using the correct seed ensures that your results are reproducible when we rerun your notebook.

In [8]:
import torch
import random
import numpy as np

def set_seed(seed: int = 42):
    random.seed(seed)  # Python's built-in random module
    np.random.seed(seed)  # NumPy
    torch.manual_seed(seed)  # PyTorch CPU
    torch.cuda.manual_seed(seed)  # PyTorch GPU (if available)
    torch.cuda.manual_seed_all(seed)  # Multi-GPU
    torch.backends.cudnn.deterministic = True  # Ensure deterministic behavior
    torch.backends.cudnn.benchmark = False  # Disable benchmark mode for reproducibility

### SET YOUR SEED TO YOUR STUDENT NUMBER HERE
my_student_number = 2554800
set_seed(my_student_number)

# Setup: loading the data

Let's make a folder to save the output of your work:

In [9]:
import os
import pandas as pd

#os.mkdir('./outputs')

Now, let's load some more packages we will need later:

In [10]:
%load_ext autoreload
%autoreload 2

# Use HuggingFace's datasets library to access the Emotion dataset
from datasets import load_dataset
import numpy as np
from sklearn.model_selection import train_test_split 


The dataset classifies paragraphs taken from corporate disclosures that discuss climate-related issues. It classifiers them into "risk" (0), "neutral" (1) or "opportunity" (2) representing the sentiment of the paragraph.

First we need to load the data. The data is already split into train, validation and test. The _validation_ set (also called 'development' set or 'devset') can be used to compute performance of your model when tuning hyperparameters, optimising combinations of features, or looking at the errors your model makes before improving it. This allows you to hold out the test set (i.e., not to look at it at all when developing your method) to give a fair evaluation of the model and how well it generalises to new examples. This avoids tuning the model to specific examples in the test set. An alternative approach to validation is to not use a single fixed validation set, but instead use [cross validation](https://scikit-learn.org/stable/modules/cross_validation.html). 

In [11]:
cache_dir = "./data_cache"

# load the original training set from HuggingFace
train_dataset = load_dataset(
    "climatebert/climate_sentiment",
    split="train",
    cache_dir=cache_dir,
)

# we're going to create a new validation set by splitting the data
dataset_splits = train_dataset.train_test_split(test_size=0.2)
train_dataset = dataset_splits["train"]
val_dataset = dataset_splits["test"]

train_texts = np.array(train_dataset["text"])
val_texts = np.array(val_dataset["text"])

train_labels = np.array(train_dataset["label"])
val_labels = np.array(val_dataset["label"])

print(f"Training dataset with {len(train_texts)} instances loaded")
print(f"Development/validation dataset with {len(val_texts)} instances loaded")

### DO NOT MODIFY
# save gold labels to file
pd.DataFrame(val_labels).to_csv('./outputs/val_labels.csv')

Training dataset with 800 instances loaded
Development/validation dataset with 200 instances loaded


In this notebook, you're going to build three different classifiers for this dataset, then compare how they work, and analyse the results. We are going to start by implementing a naïve Bayes classifier from scratch. 

We are going to begin by initialising some useful variables and doing some very simple pre-processing using CountVectorizer.

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk import word_tokenize

num_classes = 3

def preprocess(train_texts):
    vectorizer = CountVectorizer(ngram_range=(2,2), tokenizer=word_tokenize)
    X = vectorizer.fit_transform(train_texts).toarray()
    num_features = X.shape[1]

    X_val = vectorizer.transform(val_texts).toarray()

    return X, X_val, vectorizer, num_features

X, X_val, vectorizer, num_features = preprocess(train_texts)

c:\Users\arpan\anaconda3\envs\text_analytics\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


## TASK 1.1a

Complete the function below to compute the class priors, $p(y_n = c)$ for each class label $c$, where $y_n$ is the class label of document $n$. Do not use the Sklearn implementation to do this, but implement it yourself, e.g., using Numpy functions. The function must output the class priors as a list or Numpy array containing the probabilities. You do not need to apply any smoothing or regularisation.    (3 marks)

In [13]:
def compute_class_priors(texts, labels):
    priors = np.zeros(num_classes)

    ### WRITE YOUR ANSWER HERE
    class_counts = np.bincount(labels)
    priors = class_counts / len(labels)

    ### END OF ANSWER
    return priors


class_priors = compute_class_priors(train_texts, train_labels)
print(class_priors)

### DO NOT MODIFY
pd.DataFrame(class_priors).to_csv('./outputs/11a_class_priors.csv')

[0.34875 0.39625 0.255  ]


## TASK 1.1b

Complete the function below to extract n-gram features from the text, then compute the liklihood $p(x_{ni} = w | y_n = c)$ that the $i$ th n-gram in document $n$ is $w$, given that the class of $n$ is $c$. Again, do not use the Sklearn implementation to do this, but implement it yourself, e.g., using Numpy functions. The function must output the likelihoods as a 2D Numpy array containing probabilities. You should apply smoothing by adding counts of +1 to the counts of each feature.  (3 marks)

In [14]:
def compute_feature_likelihoods(X, labels):

    likelihoods = np.ones((num_features, num_classes))  # a 2D numpy array where you can store the likelihoods. Note that all values are initialised to one.

    ### WRITE YOUR ANSWER HERE
    for c in range(num_classes):
        # Get documents of class c
        class_mask = (labels == c)
        X_c = X[class_mask]
        
        # Sum counts for each feature (ngram) in this class
        feature_counts = X_c.sum(axis=0)  # Sum across documents → shape [features]
        
        # Total tokens in class (sum all ngram counts)
        total_tokens = X_c.sum()
        
        # Laplace smoothing: (count + 1) / (total_tokens + num_features)
        likelihoods[:, c] = (feature_counts + 1) / (total_tokens + num_features)


    ### END OF ANSWER
    return likelihoods

likelihoods = compute_feature_likelihoods(X, train_labels)

### DO NOT MODIFY
pd.DataFrame(likelihoods).to_csv('./outputs/11b_likelihoods.csv')

Now, we are going to use the code in the next cell to compute the log probabilities of each class for each text in the validation set. This code will use the previous functions you implemented, compute_class_priors and compute_feature_likelihoods. The log probabilities will be stored in the 'predictions' array.

In [15]:
from scipy.special import logsumexp


def NB_classify(class_priors, likelihoods, X_val):

    predictions = np.zeros((X_val.shape[0], num_classes))  # an empty numpy array to store the predictions in

    sum_of_log_likelihoods = X_val.dot(np.log(likelihoods))
    log_joint_prob = sum_of_log_likelihoods + np.log(class_priors)[None, :]
    for n, doc in enumerate(X_val):
        predictions[n, :] = log_joint_prob[n]
        predictions[n, :] -= logsumexp(predictions[n, :])
    return predictions

predictions = NB_classify(class_priors, likelihoods, X_val)
print(predictions)

[[-1.97627064e+01 -2.68951966e+00 -7.03297171e-02]
 [-1.13201115e+01 -1.18703507e-03 -6.74716460e+00]
 [-6.29842767e+00 -2.26693622e-03 -7.76301127e+00]
 [-7.92891022e+00 -2.44782231e-01 -1.52894231e+00]
 [-1.15057738e+01 -1.91330846e-02 -3.96641915e+00]
 [ 0.00000000e+00 -4.92425600e+01 -7.33630050e+01]
 [-1.18518537e-01 -5.78980648e+00 -2.21910759e+00]
 [-1.31378850e+01 -2.42176530e+00 -9.29563791e-02]
 [-4.87571626e+01  0.00000000e+00 -4.82190651e+01]
 [ 0.00000000e+00 -4.33284557e+01 -5.73522020e+01]
 [-2.38858667e+01 -5.75147396e-09 -1.89811969e+01]
 [-8.85679745e-03 -5.17855422e+00 -5.75038917e+00]
 [-6.73877403e-01 -1.81153508e+00 -1.11819711e+00]
 [-1.23736186e-08 -1.82157246e+01 -2.30371242e+01]
 [-2.34314261e+01 -4.37069048e-10 -2.17164202e+01]
 [-1.44950718e-11 -2.49591833e+01 -3.32419475e+01]
 [-1.30317119e+01 -1.40295807e+01 -2.99707102e-06]
 [ 0.00000000e+00 -6.95945632e+01 -8.52024489e+01]
 [-1.29822532e+01 -1.49276265e+00 -2.54573837e-01]
 [-3.31368875e+00 -3.70600410e-

Use the 'predictions' array above to compute and print the accuracy of the classifier on the validation set.   

In [16]:
from sklearn.metrics import accuracy_score

accuracy_score(val_labels, np.argmax(predictions, axis=1))

0.705

## TASK 1.1c

The simplicty of naïve Bayes means that we can quite easily interpret the model. In the code above, we used the functions you implemented, compute_feature_likelihoods and compute_class_priors, to train an NB classifier with our training set. Given this classifier, which are the five n-gram features that most strongly indicate that the document belongs to class 0? Store these features in the 'top_features' list below.    (4 marks)

In [17]:
top_features = []

### WRITE YOUR ANSWER HERE
sorted_indices = np.argsort(likelihoods[:, 0])[::-1]
top_indices = sorted_indices[:5]
top_features = vectorizer.get_feature_names_out()[top_indices].tolist()
### END OF ANSWER

### DO NOT MODIFY
print(top_features)
pd.DataFrame(top_features).to_csv('./outputs/11c_top_feats.csv')
###

['climate change', 'of the', ', and', 'in the', 'to the']


Up to this point, the classifier used bigrams features extracted using CountVectorizer with the wordnet tokenizer. 

## TASK 1.1d

Your task is to improve the naïve Bayes classifier by changing the preprocessing or features only. It is up to you to decide how many changes are needed to improve the classifier -- a single change may be enough to achieve a good result (and maximum marks) and you should only include steps that help performance. Complete the 'preprocess_improved' function below, and run the cell to compute accuracy of the improved classifier on the validation set.     (3 marks)

In [18]:
from nltk.stem import WordNetLemmatizer
import re
def preprocess_improved(train_texts):
    ### WRITE YOUR ANSWER HERE
    lemmatizer = WordNetLemmatizer()

    def custom_tokenizer(text):
        text = text.lower()  # Normalize casing
        text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
        tokens = word_tokenize(text)
        tokens = [lemmatizer.lemmatize(token) for token in tokens]  # Lemmatize
        return tokens
    
    vectorizer = CountVectorizer(
        ngram_range=(1, 2),  # Use unigrams + bigrams
        tokenizer=custom_tokenizer,
        stop_words='english',  # Remove stopwords
        min_df=2,  # Ignore rare terms (appearing in <2 docs)
        max_features=3000  # Limit feature space
    )
    
    #vectorizer = CountVectorizer(ngram_range=(2,2), tokenizer=word_tokenize)
    X = vectorizer.fit_transform(train_texts).toarray()
    num_features = X.shape[1]

    X_val = vectorizer.transform(val_texts).toarray()

    ### END OF ANSWER

    return X, X_val, vectorizer, num_features

X, X_val, vectorizer, num_features = preprocess_improved(train_texts)
class_priors = compute_class_priors(train_texts, train_labels)
likelihoods = compute_feature_likelihoods(X, train_labels)
predictions = NB_classify(class_priors, likelihoods, X_val)
predictions_nb = np.argmax(predictions, axis=1)

### DO NOT MODIFY
pd.DataFrame(predictions_nb).to_csv('./outputs/11d_improved_preds.csv')
accuracy_improved = accuracy_score(val_labels, predictions_nb)
print(accuracy_improved)
###

c:\Users\arpan\anaconda3\envs\text_analytics\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\arpan\anaconda3\envs\text_analytics\Lib\site-packages\sklearn\feature_extraction\text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'u', 'wa'] not in stop_words.
  warnings.warn(


0.795


## TASK 1.2

Below is an implementation of a neural network classifier that we can apply to the same dataset. However, there are some mistakes in the code and some poor choices in the choice of parameters and architecture. Your task is to fix the errors, make better parameter choices, and improve the model's performance. **Modify the code within the next cell** to improve the neural network classifier, then run it and compute its accuracy using the code in the cell after that.   (8 marks)

In [19]:
from torch import nn, optim
from transformers import AutoTokenizer
from torch.utils.data import DataLoader
import numpy as np

# Initialize tokenizer with a more suitable model
tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")  # More general-purpose than climate-specific

# Improved hyperparameters
sequence_length = 128  # Longer sequence to capture context
batch_size = 32        # Larger batch size for stable training
embedding_size = 128   # Larger embedding dimensions
hidden_size = 64       # Balanced hidden layer size

def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=sequence_length,
        return_tensors="pt"
    )

# Tokenize datasets
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_train.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
train_loader = DataLoader(tokenized_train, batch_size=batch_size, shuffle=True)

tokenized_val = val_dataset.map(tokenize_function, batched=True)
tokenized_val.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
val_loader = DataLoader(tokenized_val, batch_size=batch_size, shuffle=False)

class FFTextClassifier(nn.Module):
    def __init__(self, vocab_size, sequence_length, num_classes, embedding_size=128, hidden_size=64):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_size)
        self.dropout = nn.Dropout(0.3)  # Regularization
        self.fc1 = nn.Linear(embedding_size * sequence_length, hidden_size)
        self.fc2 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()

    def forward(self, input_ids):
        x = self.embedding(input_ids)  # (batch, seq_len, emb_size)
        x = x.view(x.size(0), -1)      # Flatten: (batch, seq_len*emb_size)
        x = self.dropout(x)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

    def run_training(self, train_loader, val_loader, num_epochs=5, lr=1e-4):
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.AdamW(self.parameters(), lr=lr, weight_decay=1e-5)  # AdamW with L2 regularization
        
        for epoch in range(num_epochs):
            self.train()
            train_loss = 0.0
            for batch in train_loader:
                optimizer.zero_grad()
                outputs = self(batch['input_ids'])
                loss = criterion(outputs, batch['label'])
                loss.backward()
                optimizer.step()
                train_loss += loss.item()
            
            # Validation
            self.eval()
            val_loss = 0.0
            with torch.no_grad():
                for batch in val_loader:
                    outputs = self(batch['input_ids'])
                    val_loss += criterion(outputs, batch['label']).item()
            
            print(f"Epoch {epoch+1}/{num_epochs} | Train Loss: {train_loss/len(train_loader):.4f} | Val Loss: {val_loss/len(val_loader):.4f}")

def predict_nn(trained_model, data_loader):

    trained_model.eval()

    pred_labs = []  # predicted labels to return
    
    for batch in data_loader:
        test_output = trained_model(batch['input_ids'])
        predicted_labels = test_output.argmax(1)
        pred_labs.extend(predicted_labels.tolist())
    
    return pred_labs

# Initialize and train
vocab_size = tokenizer.vocab_size
model = FFTextClassifier(vocab_size, sequence_length, num_classes, embedding_size, hidden_size)
model.run_training(train_loader, val_loader)

predictions_nn = predict_nn(model, val_loader)
# Predict and evaluate
### DO NOT MODIFY
pd.DataFrame(predictions_nn).to_csv("./outputs/12_nn_preds.csv")
accuracy_nn = accuracy_score(val_labels, predictions_nn)
print(accuracy_nn)
###

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Epoch 1/5 | Train Loss: 1.0758 | Val Loss: 1.0574
Epoch 2/5 | Train Loss: 0.8226 | Val Loss: 1.1288
Epoch 3/5 | Train Loss: 0.6620 | Val Loss: 1.2055
Epoch 4/5 | Train Loss: 0.4418 | Val Loss: 1.0702
Epoch 5/5 | Train Loss: 0.3116 | Val Loss: 1.1176
0.395


In [20]:
### DO NOT MODIFY
set_seed(2554800)
###

### WRITE YOUR ANSWER HERE: MODIFY THE CODE WITHIN THIS CELL
 
from torch import nn
from torch import optim
from transformers import AutoTokenizer
from torch.utils.data import DataLoader

tokenizer = AutoTokenizer.from_pretrained("climatebert/distilroberta-base-climate-f")  

sequence_length = 5  # truncate all docs longer than this. Pad all docs shorter than this.
batch_size = 1

def tokenize_function(examples):
    return tokenizer(
        examples["text"],  # Adjust the key based on your dataset structure
        padding="max_length",  # Ensures equal sequence lengths
        truncation=True,       # Truncates longer sequences
        max_length=sequence_length,        # Adjust as needed
        return_tensors="pt"
    )

tokenized_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])  # Adjust column names
train_loader = DataLoader(tokenized_dataset, batch_size=batch_size, shuffle=True)

tokenized_dataset = val_dataset.map(tokenize_function, batched=True)
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])  # Adjust column names
val_loader = DataLoader(tokenized_dataset, batch_size=batch_size, shuffle=False)

class FFTextClassifier(nn.Module):
    
    def __init__(self, vocab_size, sequence_length, num_classes, embedding_size=2, hidden_size=2):
        super(FFTextClassifier, self).__init__()

        self.embedding_size = embedding_size
        self.sequence_length = sequence_length

        # Here we just need to construct the components of our network. We don't need to connect them together yet.
        self.embedding_layer = nn.Embedding(vocab_size, embedding_size) # embedding layer
        self.hidden_layer = nn.Linear(self.embedding_size*sequence_length, hidden_size)  #nn.LSTM(self.embedding_size, hidden_size, bidirectional=True, batch_first=True) # Hidden layer
        self.activation = nn.ReLU() # Hidden layer
        self.output_layer = nn.Linear(hidden_size, num_classes) # Full connection layer
        
        
    def forward(self, input_words):
        # Input dimensions are:  (batch_size, seq_length)
        embedded_words = self.embedding_layer(input_words)  # (batch_size, seq_length, embedding_size)

        # flatten the sequence of embedding vectors for each document into a single vector.
        embedded_words = embedded_words.reshape(embedded_words.shape[0], self.sequence_length*self.embedding_size)  #(embedded_words.shape[0], self.sequence_length*self.embedding_size)  # batch_size, seq_length*embedding_size

        z = self.hidden_layer(embedded_words)   # (batch_size, seq_length, hidden_size)
        h = self.activation(z)                  # (batch_size, seq_length, hidden_size)

        output = self.output_layer(h)                      # (batch_size, num_classes)

        # Notice we haven't applied a softmax activation to the output layer -- it's not required by Pytorch's loss function.

        return output


    def run_training(self, train_dataloader, dev_dataloader):

        # training hyperparameters
        num_epochs = 1   
        learning_rate = 5e5  # learning rate for the gradient descent optimizer, related to the step size

        loss_fn = nn.CrossEntropyLoss()  # create loss function object
        optimizer = optim.Adam(self.parameters(), lr=learning_rate)  # create the optimizer
        
        dev_losses = []
            
        for e in range(num_epochs):
            # Track performance on the training set as we are learning...
            train_losses = []

            self.train()  # Put the model in training mode.

            for i, batch in enumerate(train_dataloader):
                # Iterate over each batch of data

                optimizer.zero_grad()  # Reset the optimizer

                # Use the model to perform forward inference on the input data.
                # This will run the forward() function.
                output = self(batch['input_ids'])

                # Compute the loss for the current batch of data
                batch_loss = loss_fn(output, batch['label'])

                # Perform back propagation to compute the gradients with respect to each weight
                batch_loss.backward()

                # Update the weights using the compute gradients
                optimizer.step()

                # Record the loss from this sample to keep track of progress.
                train_losses.append(batch_loss.item())

            print("Epoch: {}/{}".format((e+1), num_epochs),
                "Training Loss: {:.4f}".format(np.mean(train_losses)))

            self.eval()  # Switch model to evaluation mode

            dev_losses_epoch = []
            
            for dev_batch in dev_dataloader:
                dev_output = self(dev_batch['input_ids'])
                dev_loss = loss_fn(dev_output, dev_batch['label'])

                # Save the loss on the dev set
                dev_losses_epoch.append(dev_loss.item())
                        
            dev_losses.append(np.mean(dev_losses_epoch))
                    
            print("Epoch: {}/{}".format((e+1), num_epochs),
                "Validation Loss: {:.4f}".format(dev_losses[-1]) )

        return dev_losses

def predict_nn(trained_model, data_loader):

    trained_model.eval()

    pred_labs = []  # predicted labels to return
    
    for batch in data_loader:
        test_output = trained_model(batch['input_ids'])
        predicted_labels = test_output.argmax(1)
        pred_labs.extend(predicted_labels.tolist())
    
    return pred_labs

vocab_size = max(tokenizer.get_vocab().values()) + 1
nn_classifier_model = FFTextClassifier(vocab_size, sequence_length, num_classes)
dev_losses = nn_classifier_model.run_training(train_loader, val_loader)

predictions_nn = predict_nn(nn_classifier_model, val_loader)

### END OF ANSWER 

### DO NOT MODIFY
pd.DataFrame(predictions_nn).to_csv("./outputs/12_nn_preds.csv")
accuracy_nn = accuracy_score(val_labels, predictions_nn)
print(accuracy_nn)
###

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Epoch: 1/1 Training Loss: 17603154118783900.0000
Epoch: 1/1 Validation Loss: 116634.2625
0.455


We now explore the use of transformers for building a text classifier. First, let's look at how the process a document. We'll chose one at random from the training set:

In [21]:
### DO NOT MODIFY
chosen_document = train_texts[np.random.randint(len(train_texts))]

## TASK 1.3a

Use the HuggingFace transformers library to load the pretrained BERT model "prajjwal1/bert-tiny". Obtain a document embedding for the chosen document given above. Comment your code to explain how it obtains a representation of the document.    (3 marks)

In [26]:
### DO NOT MODIFY
set_seed(2554800)
###

### WRITE YOUR ANSWER HERE
from transformers import AutoTokenizer, AutoModel
import torch
# Load the pretrained tiny BERT model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("prajjwal1/bert-tiny")
model = AutoModel.from_pretrained("prajjwal1/bert-tiny")

# Choose a document (using first validation text as example)
document = chosen_document  # Replace with your chosen document if needed

# Tokenize the document and prepare for model input
inputs = tokenizer(
    document,
    padding=True,          # Pad to max length
    truncation=True,       # Truncate if needed
    max_length=512,        # BERT's max input length
    return_tensors="pt"    # Return PyTorch tensors
)

# Get model outputs (no gradient needed for inference)
with torch.no_grad():
    outputs = model(**inputs)

# Extract embeddings:
# We use mean pooling of last hidden states to get a fixed-size doc embedding
# Shape of last_hidden_states: [batch_size, seq_length, hidden_dim]
last_hidden_states = outputs.last_hidden_state

# Mean pooling across sequence dimension (axis=1) to get document embedding
doc_emb = last_hidden_states.mean(dim=1).squeeze().tolist()
#doc_emb = []  # use this variable to store the document embedding


### END OF ANSWER

### DO NOT MODIFY
pd.DataFrame(doc_emb).to_csv('./outputs/13a_sen_emb.csv')
print(doc_emb)
###

[0.008394082076847553, 0.9726738333702087, -0.9660142660140991, -0.5269185304641724, 0.22151845693588257, -0.06738971918821335, 0.36344945430755615, 0.7301638126373291, -1.1319618225097656, 0.019217297434806824, 0.64316725730896, 1.0371851921081543, -0.6985160708427429, -1.2869834899902344, 1.7348040342330933, -0.9035645723342896, -1.0529977083206177, -0.701332688331604, -0.43761876225471497, 0.8589183688163757, 0.27598750591278076, 0.05708514153957367, 0.9188026189804077, 1.3096586465835571, 2.509965419769287, -0.5023406147956848, 0.0976107120513916, -0.17974132299423218, -0.40038734674453735, -0.0323413722217083, -0.46496328711509705, -2.2116446495056152, 0.15867452323436737, -0.4555830657482147, 0.955571711063385, -1.5826778411865234, -1.0851718187332153, 0.25275203585624695, -1.504386305809021, -0.24071376025676727, 0.6129204034805298, 0.16323818266391754, -0.43764105439186096, -1.6788743734359741, 0.2927488684654236, -1.4292832612991333, 0.2150164395570755, 0.42032426595687866, -0

## TASK 1.3b

Using the same document embeddings method as the previous task, find the most similar document to the 'chosen_document' from within the validation set (from the 'val_texts' object). Use a standard similarity metric that considers the direction but not the magnitude of the embedding vectors. Use the same model as in task 1.8.  (2 marks)

In [27]:
### DO NOT MODIFY
set_seed(2554800)
###

### WRITE YOUR ANSWER HERE
from sklearn.metrics.pairwise import cosine_similarity
# Define embedding function (reusable)
def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

# Get embedding for the chosen document (replace with your actual chosen_document)
#chosen_document = chosen_document  # Example - use your specific document
chosen_emb = get_embedding(chosen_document)

# Compute embeddings for all validation documents
val_embeddings = np.array([get_embedding(doc) for doc in val_texts])

# Calculate cosine similarities (measures directional similarity, ignores magnitude)
similarities = cosine_similarity([chosen_emb], val_embeddings)[0]

# Find most similar document (excluding identical matches)
most_similar_doc = np.argmax(similarities)
#most_similar_doc = ""  # use this variable to store the most similar document

### END OF ANSWER

### DO NOT MODIFY
pd.DataFrame([chosen_document, most_similar_doc]).to_csv("./outputs/13b_most_similar.csv")
print(chosen_document)
print(most_similar_doc)
###

We strengthened our commitment to responsible ownership by investing directly in projects and companies that are making a positive impact, such as a $170 million commitment to affordable housing. And through the development of our new Climate Change Portfolio Transition Plan, we’ve committed to transitioning our investment portfolio to carbon neutrality, achieving net zero emissions by 2050.
0


## TASK 1.3c

Implement a classifier based on the same pretrained transformer model, "prajjwal1/bert-tiny". Evaluate your model's performance on the validation set. Use an 'auto class' from HuggingFace to build your classifier (see https://huggingface.co/docs/transformers/model_doc/auto).   (6 marks)

In [29]:
### DO NOT MODIFY
set_seed(2554800)
###

### WRITE YOUR ANSWER HERE
from transformers import AutoModelForSequenceClassification
from tqdm import tqdm

model_name = "prajjwal1/bert-tiny"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_classes  # Set based on your dataset (3 for climate sentiment)
)

# Tokenize validation set
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=128,
        return_tensors="pt"
    )

tokenized_val = val_dataset.map(tokenize_function, batched=True)
tokenized_val.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

# Create DataLoader
val_loader = DataLoader(tokenized_val, batch_size=16)

# Prediction loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()
predictions_bert = []  # use this variable to store the predicted labels for the validation set
with torch.no_grad():
    for batch in tqdm(val_loader, desc="Evaluating"):
        inputs = {
            "input_ids": batch["input_ids"].to(device),
            "attention_mask": batch["attention_mask"].to(device)
        }
        outputs = model(**inputs)
        preds = torch.argmax(outputs.logits, dim=1)
        predictions_bert.extend(preds.cpu().tolist())

### END OF ANSWER

### DO NOT MODIFY
pd.DataFrame(predictions_bert).to_csv('./outputs/13c_bert_preds.csv')
accuracy_tinybert = accuracy_score(val_dataset["label"], predictions_bert)
print(accuracy_tinybert)
###

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Evaluating: 100%|██████████| 13/13 [00:00<00:00, 18.59it/s]

0.38


In [28]:
### DO NOT MODIFY
set_seed(2554800)
###

### WRITE YOUR ANSWER HERE

from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_metric
import numpy as np

# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("prajjwal1/bert-tiny")
model = AutoModelForSequenceClassification.from_pretrained("prajjwal1/bert-tiny", num_labels=3)

# Tokenization function
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=128  # Optimized for bert-tiny
    )

# Tokenize datasets
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_val = val_dataset.map(tokenize_function, batched=True)

# Set PyTorch format
tokenized_train.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
tokenized_val.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

# Training configuration
training_args = TrainingArguments(
    output_dir="./bert_results",
    num_train_epochs=3,  # Sufficient for fine-tuning
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=2e-5,  # Standard for transformer fine-tuning
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

# Metrics calculation
def compute_metrics(eval_pred):
    metric = load_metric("accuracy")
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    compute_metrics=compute_metrics,
)

# Fine-tune the model
trainer.train()

# Generate predictions
predictions = trainer.predict(tokenized_val)
predictions_bert = np.argmax(predictions.predictions, axis=1).tolist()
### END OF ANSWER

### DO NOT MODIFY
pd.DataFrame(predictions_bert).to_csv('./outputs/13c_bert_preds.csv')
accuracy_tinybert = accuracy_score(val_dataset["label"], predictions_bert)
print(accuracy_tinybert)
###

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/200 [00:00<?, ? examples/s]

c:\Users\arpan\anaconda3\envs\text_analytics\Lib\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.044957,0.550000
2,No log,1.031114,0.565000
3,No log,1.024307,0.590000


0.59
